In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
import pandas as pd
import numpy as np

imdb_data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/ai_school/data/IMDB Dataset.csv")
# print(imdb_data)

In [4]:
# pos, neg 값을 숫자로 변형
imdb_data['sentiment'] = imdb_data['sentiment'].replace("positive", 1)
# 위는 'imdb_data['sentiment'].replace("positive", 1, inplace=True)'와 같음
imdb_data['sentiment'] = imdb_data['sentiment'].replace("negative", 0)
# print(imdb_data)

In [5]:
# 단어 아니면 삭제 (정규 표현식 사용)
imdb_data['review'] = imdb_data['review'].str.replace("[^\w]|br", " ")
print(imdb_data)

                                                  review  sentiment
0      One of the other reviewers has mentioned that ...          1
1      A wonderful little production            The f...          1
2      I thought this was a wonderful way to spend ti...          1
3      Basically there s a family where a little boy ...          0
4      Petter Mattei s  Love in the Time of Money  is...          1
...                                                  ...        ...
49995  I thought this movie did a down right good job...          1
49996  Bad plot  bad dialogue  bad acting  idiotic di...          0
49997  I am a Catholic taught in parochial elementary...          0
49998  I m going to have to disagree with the previou...          0
49999  No one expects the Star Trek movies to be high...          0

[50000 rows x 2 columns]


In [6]:
# 혹시 공백이 있으면 null array로 (ex : 이모티콘만 사용해서 다 지워진 행)
imdb_data['review'] = imdb_data['review'].replace('', np.nan)
imdb_data['sentiment'] = imdb_data['sentiment'].replace('', np.nan)
# null array 모두 제거 (공백인 열 모두 제거)
imdb_data = imdb_data.dropna(how='any')

print("# preproecssing done") # 눈으로 진행도를 확인하기 위한 출력 (필수 X))

# preproecssing done


In [7]:
from sklearn.model_selection import train_test_split

review_train, review_test, y_train, y_test = train_test_split(imdb_data['review'], imdb_data['sentiment'], test_size=0.25, shuffle=False, random_state=1004)

print('# split done')

# split done


In [8]:
# stopwords 지정 (예시)
stopwords = ['a', 'an']

# 토큰화 진행 (시간 관계상 스플릿으로 진행, 더 높은 정확도를 원한다면 토크나이저 사용)
X_train = []
for stc in review_train:
    token = []
    words = stc.split()
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_train.append(token)

X_test = []
for stc in review_test:
    token = []
    words = stc.split()
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_test.append(token)

print('# tokenization done')

# tokenization done


In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer

# X_train 단어들을 토대로 정수 인덱스 설정 (단어를 정수형태로 변환), 전체 단어 갯수 설정
# tensorflow의 임베딩은 무조건 정수 형태로만 input을 받는다.
# 왜 트레인셋만?.. 원한다면 처음에 test/train 스플릿하기 전에, 전처리해서 fit해도 무방
# 왜 5000?.. 유의미한 단어 갯수를 생각해보자! 
# 빈도수가 1~2개인 단어 버려도 큰 영향을 끼치지 않을것 -> count함수써서 빈도수 낮은 것들을 버리고, 남은 단어의 갯수들
# 즉, 빈도수 1개다 -> 버리자, 빈도수 2개다 -> 버리자
tokenizer = Tokenizer(5000) # 토크나이저이 인자값을 넣으면 그 만큼만 인덱스를 생성하고 나머지는 잘라버림

# train 을 기준으로 단어마다의 인덱스를 부여
tokenizer.fit_on_texts(X_train) #  X_train을 스플릿 하기 전에 토큰화 한 것으로 넣어도 가능 

In [10]:
# 위에서 설정된 정수 인덱스를 토대로 변환
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

print('# int_encoding done')

# int_encoding done


In [11]:
print(X_train[1])
print(y_train[1])

[106, 387, 118, 357, 1, 1364, 3016, 5, 51, 51, 158, 54, 2198, 1550, 2, 409, 2, 527, 277, 3, 1847, 4, 1, 444, 408, 1, 150, 24, 566, 67, 2206, 494, 4098, 22, 60, 44, 192, 29, 1, 17, 23, 44, 29, 1, 2316, 176, 3408, 95, 19, 48, 371, 61, 1, 795, 32, 1, 1848, 4, 1798, 22, 60, 5, 6, 67, 273, 1, 147, 17, 6, 5, 412, 2, 2441, 408, 106, 4391, 357, 42, 27, 3, 1, 79, 1146, 11, 3, 201, 2, 25, 113, 1, 1847, 63, 267, 348, 15, 1, 118, 179, 1, 1030, 3, 1, 2791, 59, 249, 72, 356, 1, 2210, 960, 3198, 1249, 1202, 89, 4982, 6, 304, 21, 256, 1864, 2, 256, 4320, 572, 15, 1, 134, 3643, 2, 2, 1, 712, 572, 3, 64, 1047, 15, 11, 171, 2348, 24, 2010, 67, 218]
1


In [12]:
# 문장(데이터) 길이를 맞춰준다 
# 임의로 맞추는게 아니고, 데이터셋을 보면서 최대 문장의 길이가 얼마인지 확인하고 거기에 맞춰서
max_len = 500 
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [13]:
print(X_train[1])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [14]:
# 레이어들을 쌓을 모델을 생성
model = Sequential()
# 단어를 임베딩하는데, 5000개의 단어를 120차원으로 내보내겠다
# Emdedding의 인자값 : 1인자 = 내가 넣을 단어의 갯수 (총 인덱스의 갯수), 2인자 = 출력할 차원 (덴스 벡터의 차원), 3인자 = 문장의 길이
model.add(Embedding(5000, 120)) # Embedding의 인자값에 위에서 설정한 max_len인 'input_length = 500' 이 생략된 상태
# RNN - simpleRNN / LSTM
model.add(LSTM(120))
# 긍정/부정을 판단하니까 이진 분류 -> sigmoid 함수 사용
model.add(Dense(1, activation='sigmoid'))

In [15]:
# 혹시 5회 이상 검증데이터 loss가 증가하면, 과적합될 수 있으므로 학습을 조기종료!
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
# 훈련을 거듭하면서, 가장 검증데이터 정확도가 높았던 순간을 체크포인트로 저장
model_check = ModelCheckpoint('the_best.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [16]:
# 긍정/부정을 판단하니까 손실함수는 이진 교차 엔트로피, 최적화는 adam, 평가 기준은 acc (출력할때 뜬다)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1, batch_size=64, callbacks=[early_stop, model_check])

586/586 [==============================] - ETA: 0s - loss: 0.3987 - acc: 0.8154
Epoch 00001: val_acc improved from -inf to 0.82968, saving model to the_best.h5
586/586 [==============================] - 534s 910ms/step - loss: 0.3987 - acc: 0.8154 - val_loss: 0.3990 - val_acc: 0.8297


In [17]:
# 정확도 측정 / 출력하면 [loss, acc]
print(model.evaluate(X_test, y_test))
# X 값은 전처리, 토큰화, 정수인코딩이 된 상태의 문장이어야 한다
# print(model.predict(X))

391/391 [==============================] - 42s 108ms/step - loss: 0.3990 - acc: 0.8297
[0.39899691939353943, 0.8296800255775452]
